In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
# lib for modelling
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score,roc_auc_score,roc_curve,confusion_matrix,classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
# import EDA library
from pandas_profiling import ProfileReport

In [ ]:
# loading the dataset 
df=pd.read_csv('../input/predicting-churn-for-bank-customers/Churn_Modelling.csv')

In [ ]:
# generating report for the data for the better visualization
ProfileReport(df)

In [ ]:
# lets look for the column in data 
df.head(5)

In [ ]:
# Data pre - processing
df.drop(['CustomerId','Surname'],axis=1,inplace=True) 
### let's drop few columns which won't contribute to the model as we have seen above in report

In [ ]:
#  one hot encoding for the column `Geo` 
Geography_dummies = pd.get_dummies(prefix='Geo',data=df,columns=['Geography'])

In [ ]:
# label encoding for the gender column  as its just male and female we have not done on hot as its nor ordinal data
Gender_dummies = Geography_dummies.replace(to_replace={'Gender': {'Female': 1,'Male':0}})

In [ ]:
df_new = Gender_dummies
df_new.head()

In [ ]:
# Doing EDA for the dataset since pandas profiling had done our most of the work but still we need to work out
# Check the distribution of y variable to see if it's a case of unbalanced class

sns.countplot(y=df_new.Exited ,data=df_new)
plt.xlabel("Count of each Target class")
plt.ylabel("Target classes")
plt.show()

In [ ]:
# Check the distribution of all the features

df_new.hist(figsize=(15,12),bins = 15)
plt.title("Features Distribution")
plt.show()

# We are not checking for the multicollinearity as we have already looked the same in profile report

In [ ]:
# as we have seen that there is imbalanced in the dataset for the target hence we need to balance the data
# Split the y variable series and x variables dataset

X = df_new.drop(['Exited'],axis=1)
y = df_new.Exited
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=0)
X, y = smt.fit_resample(X, y)
print(X.shape)
print(y.shape)

In [ ]:
# Splitting the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [ ]:
# using power transfor to normalise the data
from sklearn.preprocessing import PowerTransformer
pt=PowerTransformer()
Xpowertrain=pt.fit_transform(X_train)
xpowertest=pt.transform(X_test)

In [ ]:
# apply logit model
model=sm.Logit(y,sm.add_constant(X)).fit()
print(model.summary())

# As per the model location gender has lesser pvalue but we can not drop it as per domain knowledge as we need it identify for the location and gender as a factor to churn

In [ ]:
lr=LogisticRegression()

In [ ]:
lr.fit(Xpowertrain,y_train)

In [ ]:
lr.score(Xpowertrain,y_train)

In [ ]:
lr.score(xpowertest,y_test)

In [ ]:
# Predicting target values by using x_test and our model:
y_pred = lr.predict(xpowertest)


# Model evalution metric

In [ ]:
print ('Confusion Matrix :')
print(confusion_matrix(y_test, y_pred))
print('Accuracy Score :',accuracy_score(y_test, y_pred) )
print('Recall Score :',recall_score(y_test, y_pred) )
print('Precision Score :',precision_score(y_test, y_pred) )
print('Report : ')
print(classification_report(y_test, y_pred))

      

In [ ]:
# ROC AUC curve
y_pred_prob1 = lr.predict_proba(xpowertest)[:,1]
fpr1 , tpr1, thresholds1 = roc_curve(y_test, y_pred_prob1)

In [ ]:
plt.plot(fpr1, tpr1, label= "Logistic Regression")
plt.legend()
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title('ROC-AUC curve')
plt.show()